# Лаб-3. Рекомендательные системы

In [1]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

import pandas as pd

In [6]:
# Чтение файла .tsv
data = pd.read_csv('ml-latest-small/title.ratings.tsv', sep='\t')

# Вывод первых строк
print(data.tail())

# Условие для фильтрации
tconst_value = 'tt0114709'
result = data[data['tconst'] == tconst_value]

# Вывод результата
print(result)

            tconst  averageRating  numVotes
1506745  tt9916730            7.0        12
1506746  tt9916766            7.1        24
1506747  tt9916778            7.2        37
1506748  tt9916840            6.9        11
1506749  tt9916880            7.9         9
          tconst  averageRating  numVotes
87209  tt0114709            8.3   1097902


In [7]:
# Выбираем девайс
device = "cpu" if torch.cuda.is_available() else "cpu"
print(f'Device: {device}')

Device: cpu


В качестве датасета будем использовать MovieLens

https://grouplens.org/datasets/movielens/

А именно, самый маленький вариант со 100 тыс. оценок

https://files.grouplens.org/datasets/movielens/ml-latest-small.zip


In [12]:
# Для загрузки датасета напишем свою реализацию класса Dataset
class MovielensDataset(Dataset):
    r"""seed должен быть одинаковым для обучающей и тренировочной выборки"""
    def __init__(self, source, train=True, seed=1):
        ratings      = pd.read_csv(rf"{source}\ratings.csv")
        self.movies  = pd.read_csv(rf"{source}\movies.csv")

        # Преобразовываем Id фильмов в индексы в таблице movies
        x = self.movies.loc[:,['movieId']]
        x['movieId'], x.index = x.index, x['movieId'].values
        ratings['movieId'] = ratings['movieId'].map(x.to_dict()['movieId'])

        # делим датасет 80% на 20%
        train_data = ratings.sample(frac=0.8, random_state=seed)
        test_data  = ratings.drop(train_data.index)

        self.ratings = train_data if train else test_data

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        sample = self.ratings.iloc[idx]
        return {
            "user": torch.LongTensor([sample['userId']]),
            "movie": torch.LongTensor([sample['movieId']]),
            "rating": torch.FloatTensor([sample['rating']])
        }
    def add_new_user(self, movie_ids, ratings, seed=1):

        # Проверяем, что количество фильмов совпадает с количеством оценок
        if len(movie_ids) != len(ratings):
            raise ValueError("Количество фильмов и оценок должно совпадать.")
        
        # Уникальный идентификатор для нового пользователя
        new_user_id = self.ratings['userId'].max() + 1
        
        # Создаем временную метку
        timestamp = int(pd.Timestamp.now().timestamp())
        
        # Создаем данные для нового пользователя
        new_user_ratings = pd.DataFrame({
            'userId': [new_user_id] * len(movie_ids),
            'movieId': movie_ids,
            'rating': ratings,
            'timestamp': [timestamp] * len(movie_ids)
        })
        
        # Добавляем данные нового пользователя в рейтинг
        self.ratings = pd.concat([self.ratings, new_user_ratings], ignore_index=True)
        
        # Возвращаем новый идентификатор пользователя
        return new_user_id


batch_size = 200

sataset_source = r'.\ml-latest-small'

movielens_train = MovielensDataset(sataset_source, train=True)
movielens_test  = MovielensDataset(sataset_source, train=False)

print("До добавления нового пользователя:")
print(movielens_train.ratings['userId'].min(), movielens_train.ratings['userId'].max())
print(movielens_train.ratings['movieId'].min(), movielens_train.ratings['movieId'].max())
print(movielens_train.ratings.sort_values(by='userId').reset_index(drop=True))

ids = [1, 3, 4, 5, 7, 11, 12, 18, 19, 20, 29, 32, 66, 76, 103, 160, 172, 173, 198, 208]
rts = [4.5, 3.0, 4.0, 3.5, 5.0, 4.0, 3.5, 4.0, 4.5, 3.0, 5.0, 4.5, 4.0, 3.5, 4.0, 5.0, 3.5, 4.0, 4.5, 4.0]

# Добавление нового пользователя
new_user_id = movielens_train.add_new_user(
    movie_ids=ids,
    ratings=rts
)
print("После добавления нового пользователя:")
print(movielens_train.ratings['userId'].min(), movielens_train.ratings['userId'].max())
print(movielens_train.ratings['movieId'].min(), movielens_train.ratings['movieId'].max())
print(movielens_train.ratings.sort_values(by='userId').reset_index(drop=True))

# Проверим, добавился ли новый пользователь
new_user_id = movielens_train.ratings['userId'].max()  # Это должен быть ID нового пользователя

# Проверим, есть ли оценки для нового пользователя
new_user_ratings = movielens_train.ratings[movielens_train.ratings['userId'] == new_user_id]

print(f"Новый пользователь с ID {new_user_id} добавлен!")
print("Оценки нового пользователя:")
print(new_user_ratings)

train_loader = DataLoader(movielens_train, batch_size, True)
test_loader = DataLoader(movielens_test, batch_size, True)

До добавления нового пользователя:
1 610
0 9741
       userId  movieId  rating   timestamp
0           1     1319     4.0   964981230
1           1      801     5.0   964982400
2           1     2608     4.0   964981775
3           1      981     5.0   964982703
4           1      275     3.0   964982310
...       ...      ...     ...         ...
80664     610     8686     4.0  1479542606
80665     610     9389     3.5  1493848789
80666     610     2970     2.0  1493849562
80667     610     2916     4.5  1493847010
80668     610     5640     5.0  1479545132

[80669 rows x 4 columns]
После добавления нового пользователя:
1 611
0 9741
       userId  movieId  rating   timestamp
0           1      801     5.0   964982400
1           1     2802     4.0   964980694
2           1     2765     5.0   964981909
3           1      136     5.0   964983650
4           1      787     3.0   964982903
...       ...      ...     ...         ...
80684     611        4     4.0  1733234572
80685     611  

In [13]:
for batch in train_loader:
    for k, v in batch.items():
        print(k, v.shape)
    break

user torch.Size([200, 1])
movie torch.Size([200, 1])
rating torch.Size([200, 1])


In [14]:
# Функции для обучения из прошлой лабы, с учётом юзеров и айтемов
# Функции для обучения из прошлой лабы, с учётом юзеров и айтемов

def train_iteration(model, data_loader, loss_function, optimizer, sheduler):
    model.train()
    train_size = len(data_loader.dataset)
    for idx, batch in enumerate(data_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        pred = model(batch)
        loss = loss_function(pred, batch['rating'])
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if idx % 100 == 0:
            loss, current = loss.item(), (idx + 1) * batch_size
            print(f"loss: {loss:>7f}  [{current:>5d}/{train_size:>5d}]")

    scheduler.step(val_loss)

def train(model, train_loader, loss_function, optimizer, scheduler, epochs):
    for t in range(epochs):
        print(f"== Epoch {t + 1} ==")
        model.train() # Переключаем сеть в режим обучения

        total_loss = 0
        correct = 0
        total = 0

        train_size = len(train_loader.dataset)
        for idx, batch in enumerate(train_loader):
            batch = {k: v.to(device) for k, v in batch.items()}   # переносим наши данные на девайс
    
            pred = model(batch)                 # вычисляем предсказание модели
            loss = loss_function(pred, batch['rating'])   # вычисляем потери
            loss.backward()                 # запускаем подсчёт градиентов
    
            optimizer.step()                # делаем шаг градиентного спуска
            optimizer.zero_grad()           # обнуляем градиенты
    
            if idx % 100 == 0:
                
                lossa, current = loss.item(), (idx + 1) * batch_size
                print(f"loss: {lossa:>7f}  [{current:>5d}/{train_size:>5d}]")

            total_loss += loss.item()
        
        # Средние потери за эпоху
        avg_loss = total_loss / len(train_loader)
        scheduler.step(avg_loss)  # Передаем avg_loss в планировщик для корректной работы
                
        test(model, test_loader, loss_function)

#Функция для вывода статистики на тестовом датасете

def test(model, data_loader, loss_function):
    model.eval()
    test_size = len(data_loader.dataset)
    num_batches = len(data_loader)
    loss, correct = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            pred = model(batch)
            loss += loss_function(pred, batch['rating']).item()
            correct += (pred.argmax(1) == batch['rating']).type(torch.float).sum().item()

    loss /= num_batches
    correct /= test_size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {loss:>8f} \n")

## Матричные разложения

В матричных разложениях используется таблица юзеров-айтемов -- таблица, где по строкам находятся юзеры, по столбцам айтемы, на пересечениях оценка, которую поставил пользователь.

Эта таблица представляется в виде произведения двух матриц, матрицы пользователей и матрицы айтемов

![разложение](images/PQ.drawio.png)

В каждом столбце матрицы пользователей живёт вектор, соответствующий этому пользователю, в матрице айтема, соответственно, вектор айтема. Чтобы получить предсказание оценки, надо их перемножить.

Есть много разных способов находить матричные разложения, поскольку у нас тут pytorch, мы просто возьмём два `Embedding` слоя, перемножим, и скажем что это наша модель, которую обучим градиентным спуском


In [15]:
class MatrixFactorization(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

    def forward(self, batch):
        movie_emb = self.user_embeddings(batch['user'])
        user_emb = self.movie_embeddings(batch['movie'])
        return (movie_emb * user_emb).sum(2)


mf_model = MatrixFactorization().to(device)
mf_loss = nn.MSELoss()
mf_optimizer = torch.optim.SGD(mf_model.parameters(), lr=1)

train(10, mf_model, mf_loss, mf_optimizer)

TypeError: train() missing 2 required positional arguments: 'scheduler' and 'epochs'

Фактически, если к этой моделе в сумму добавить общую константу и константу для кадого пользователя и айтема, мы получим Factorization Machine

https://www.ismll.uni-hildesheim.de/pub/pdfs/Rendle2010FM.pdf

А это значит, что помимо эмбедингов с юзерами и айтемами мы можем легко добавить дополнительных параметров! (например тех, что у нас в таблице tags.csv)

## DeepFM

DeepFM это расширение обычной Factorization Machine для использования 

https://arxiv.org/pdf/1703.04247

Идея состоит в том, чтобы расположить рядом с обычной Factorization Machine нейронную сеть, которая будет параллельно существовать с матричным разложением

![DeepFM](images/DeepFM.png)

До DeepFM уже были модели, которые предварительно обучали эмбединги на матричном разложении, а потом использовали их как входные векторы сети, но тут предлагается обучать их сразу совместно

In [16]:
class DeepFM(nn.Module):
    def __init__(self):
        super().__init__()
        self.user_embeddings  = nn.Embedding(1000,  16)
        self.movie_embeddings = nn.Embedding(10000, 16)

        self.flatten = nn.Flatten()

        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
        )

        self.final_layer = nn.Linear(16*3, 1)

    def forward(self, batch):
        movie_emb = self.flatten(self.user_embeddings(batch['user']))
        user_emb  = self.flatten(self.movie_embeddings(batch['movie']))

        fm = movie_emb * user_emb

        deep = torch.cat([movie_emb, user_emb], 1)
        deep = self.deep_layers(deep)

        v = torch.cat([fm, deep], 1)
        v = self.final_layer(v)
        # делаем сигмоиду на выходе и масштабируем к оценкам от 0 до 5
        return torch.sigmoid(v) * 5


deep_mf_model = DeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.SGD(deep_mf_model.parameters(), lr=1e-1)

train(5, deep_mf_model, deep_mf_loss, deep_mf_optimizer)

TypeError: train() missing 2 required positional arguments: 'scheduler' and 'epochs'

Есть и более прокаченные версии машины факторизации на нейронках, например xDeepFM

https://arxiv.org/pdf/1803.05170

## Задание

Основное задание:
1) Достичь меньше чем 0.8 значения MSELoss на этом датасете (5 баллов)
2) МОЖНО ДЕЛАТЬ ТОЛЬКО ПОСЛЕ ТОГО КАК СДЕЛАНО ПЕРВОЕ ЗАДАНИЕ!  
    Добавить в тренировочный датасет нового пользователя - себя и дать оценки минимум 20 фильмов, обучить модель с учётом этого пользователя и сделать для себя рекомендации. (5 баллов) (пожалуйста, не дописывайте себя в файлик, сделайте пользователя добавление в питоне)

Дополнительные задания:
1) Добавить в модель использование тегов из таблички `tags.csv` (5 дополнительных баллов)
2) Добавить в модель использование дополнительных данных из источников `links.csv` (5 дополнительных баллов)

In [17]:
#улучшенная deepfm

class DeepFM(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Embeddings
        self.user_embeddings = nn.Embedding(1001, 16)
        self.movie_embeddings = nn.Embedding(10020, 16)

        # Deep layers with increased layer sizes, Dropout, and Batch Normalization
        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        # Final layer
        self.final_layer = nn.Linear(33, 1)

    def forward(self, batch):
        # Получаем эмбеддинги для пользователя и фильма и выравниваем их
        user_emb = self.user_embeddings(batch['user']).view(batch['user'].size(0), -1)  # [batch_size, 16]
        movie_emb = self.movie_embeddings(batch['movie']).view(batch['movie'].size(0), -1)  # [batch_size, 16]
    
        # Factorization Machine (FM) компонент — поэлементное умножение и сумма
        fm = (user_emb * movie_emb).sum(dim=1, keepdim=True)  # [batch_size, 1]
    
        # Deep компонент
        deep = torch.cat([user_emb, movie_emb], dim=1)  # [batch_size, 32]
        deep = self.deep_layers(deep)  # [batch_size, 32]
    
        # Убедимся, что размеры совпадают для конкатенации
        v = torch.cat([fm, deep], dim=1)  # Конкатенируем по dim=1: [batch_size, 33]
        v = self.final_layer(v)  # [batch_size, 1]
    
        # Sigmoid активация для выхода в диапазоне [0, 5]
        return torch.sigmoid(v) * 5



# Initialize model, loss, optimizer, and learning rate scheduler
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
deep_mf_model = DeepFM().to(device)
deep_mf_loss = nn.MSELoss()
deep_mf_optimizer = torch.optim.Adam(deep_mf_model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(deep_mf_optimizer, mode='min', factor=0.5, patience=2)

train(deep_mf_model, train_loader, deep_mf_loss, deep_mf_optimizer, scheduler, 15)

== Epoch 1 ==
loss: 1.958522  [  200/80689]
loss: 0.958518  [20200/80689]
loss: 0.979528  [40200/80689]
loss: 0.876545  [60200/80689]
loss: 1.130323  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.042732 

== Epoch 2 ==
loss: 1.049050  [  200/80689]
loss: 1.002605  [20200/80689]
loss: 0.895758  [40200/80689]
loss: 0.988102  [60200/80689]
loss: 0.767082  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.988196 

== Epoch 3 ==
loss: 1.066121  [  200/80689]
loss: 0.957649  [20200/80689]
loss: 1.049770  [40200/80689]
loss: 0.862593  [60200/80689]
loss: 0.930352  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.960779 

== Epoch 4 ==
loss: 1.072987  [  200/80689]
loss: 0.885884  [20200/80689]
loss: 0.898144  [40200/80689]
loss: 0.818786  [60200/80689]
loss: 0.821711  [80200/80689]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.932513 

== Epoch 5 ==
loss: 0.991483  [  200/80689]
loss: 0.944270  [20200/80689]
loss: 0.869380  [40200/80689]
loss: 0.938980  [60200/80689]
loss: 

In [106]:
def get_movie_recommendations(model, user_id, dataset, n_recommendations=10, device='cuda'):
    """
    Получить рекомендации для конкретного пользователя.

    :param model: Обученная модель
    :param user_id: ID пользователя
    :param dataset: Экземпляр датасета (например, MovielensDataset)
    :param n_recommendations: Количество рекомендаций
    :param device: Устройство ('cuda' или 'cpu')
    :return: Список фильмов с рекомендованными рейтингами
    """
    model.eval()  # Переводим модель в режим оценки
    with torch.no_grad():
        # Получаем список всех фильмов
        all_movie_ids = dataset.movies['movieId'].values
        
        # Проверяем, какие фильмы пользователь уже оценил
        rated_movies = dataset.ratings[dataset.ratings['userId'] == user_id]['movieId'].values
        
        # Фильтруем фильмы, которые пользователь еще не оценил
        unseen_movies = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movies]
        
        # Преобразуем индексы фильмов в индексы для эмбеддингов (если необходимо)
        movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(all_movie_ids)}
        unseen_movie_indices = [movie_to_idx[movie_id] for movie_id in unseen_movies]

        # Получаем эмбеддинги для пользователя
        user_tensor = torch.LongTensor([user_id] * len(unseen_movie_indices)).to(device)
        movie_tensor = torch.LongTensor(unseen_movie_indices).to(device)
        
        # Строим батч для вычисления предсказаний
        batch = {'user': user_tensor, 'movie': movie_tensor}
        
        # Прогоняем через модель для получения рейтингов
        predictions = model(batch).cpu().numpy()
        
        # Выбираем топ-N фильмов по предсказанным рейтингам
        top_n_indices = np.argsort(predictions.flatten())[-n_recommendations:][::-1]
        
        # Создаем массив идентификаторов фильмов на основе полученных индексов
        top_n_movie_ids = [unseen_movies[i] for i in top_n_indices]
        top_n_ratings = predictions.flatten()[top_n_indices]

        # Выводим рекомендации с названиями фильмов и предсказанными рейтингами
        recommended_movies = []
        for movie_id, rating in zip(top_n_movie_ids, top_n_ratings):
            movie_title = dataset.movies.loc[dataset.movies['movieId'] == movie_id, 'title']
            if movie_title.empty:
                recommended_movies.append(f"Movie ID: {movie_id} не найден в списке фильмов.")
            else:
                movie_title = movie_title.values[0]
                recommended_movies.append(f"Movie ID: {movie_id}, Title: {movie_title}, Predicted Rating: {rating:.2f}")
        
        return recommended_movies

# Пример использования:
user_id = 611  # ID пользователя, для которого генерируем рекомендации
n_recommendations = 10  # Количество рекомендаций

# Получаем рекомендации
recommended_movies = get_movie_recommendations(deep_mf_model, user_id, movielens_train, n_recommendations, device='cpu')

# Выводим рекомендации
print(f"Топ-{n_recommendations} рекомендаций для пользователя {user_id}:")
for movie in recommended_movies:
    print(movie)


Топ-10 рекомендаций для пользователя 611:
Movie ID: 694, Title: Substitute, The (1996), Predicted Rating: 4.51
Movie ID: 9618 не найден в списке фильмов.
Movie ID: 659 не найден в списке фильмов.
Movie ID: 1730, Title: Kundun (1997), Predicted Rating: 4.47
Movie ID: 277, Title: Miracle on 34th Street (1994), Predicted Rating: 4.47
Movie ID: 922, Title: Sunset Blvd. (a.k.a. Sunset Boulevard) (1950), Predicted Rating: 4.46
Movie ID: 257, Title: Just Cause (1995), Predicted Rating: 4.45
Movie ID: 3622, Title: Twelve Chairs, The (1970), Predicted Rating: 4.45
Movie ID: 6153, Title: Zapped! (1982), Predicted Rating: 4.45
Movie ID: 944, Title: Lost Horizon (1937), Predicted Rating: 4.44


## Бонусное изменение моделей

In [89]:
class newDeepFM(nn.Module):
    def __init__(self, tag_embedding_size=8, imdb_embedding_size=8, max_tags=5):
        super().__init__()

        # Embeddings
        self.user_embeddings = nn.Embedding(1001, 16)  # 1001 пользователей, 16 размерность
        self.movie_embeddings = nn.Embedding(10020, 16)  # 10020 фильмов, 16 размерность
        self.tag_embeddings = nn.Embedding(1000, tag_embedding_size, padding_idx=0)  # 1000 тегов, 8 размерность
        self.imdb_embeddings = nn.Embedding(10000, imdb_embedding_size)  # 10000 возможных IMDb ID, 8 размерность

        # Deep layers
        input_size = 56  
        self.deep_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
        )

        # Final layer
        self.final_layer = nn.Linear(33, 1)

    def forward(self, batch):
        # Embeddings
        user_emb = self.user_embeddings(batch['user']).view(batch['user'].size(0), -1)  # [batch_size, 16]
        movie_emb = self.movie_embeddings(batch['movie']).view(batch['movie'].size(0), -1)  # [batch_size, 16]
    
        # Тег
        tags = batch['tags'].clamp(0, self.tag_embeddings.num_embeddings - 1)  # Ограничиваем индексы
        tag_emb = self.tag_embeddings(tags)  # [batch_size, max_tags, tag_embedding_size]
        tag_emb = tag_emb.mean(dim=1)  # Усреднение по тегам, [batch_size, tag_embedding_size]

        # IMDb
        imdb_emb = self.imdb_embeddings(batch['imdb_info'].long()).view(batch['imdb_info'].size(0), -1)  # [batch_size, imdb_embedding_size]
    
        # FM Component
        fm = (user_emb * movie_emb).sum(dim=1, keepdim=True)  # [batch_size, 1]
    
        # Deep Component
        deep = torch.cat([user_emb, movie_emb, tag_emb, imdb_emb], dim=1)  # [batch_size, input_size]
        deep = self.deep_layers(deep)  # [batch_size, 32]
    
        # Combine FM and Deep components
        v = torch.cat([fm, deep], dim=1)  # [batch_size, 33]
        v = self.final_layer(v)  # [batch_size, 1]
    
        # Output scaled to [0, 5]
        return torch.sigmoid(v) * 5


In [95]:
import pandas as pd
import numpy as np
import torch
from collections import defaultdict

class MovielensDataset(Dataset):
    def __init__(self, source, train=True, seed=1, max_tags=5):
        # Загружаем данные
        ratings = pd.read_csv(rf"{source}\ratings.csv")
        self.movies = pd.read_csv(rf"{source}\movies.csv")
        self.tags = pd.read_csv(rf"{source}\tags.csv")
        self.links = pd.read_csv(rf"{source}\links.csv")
        self.title_ratings = pd.read_csv(rf"{source}\title.ratings.tsv", sep='\t')  # Загружаем файл title.ratings

        # Преобразуем ID фильмов в индексы
        x = self.movies.loc[:, ['movieId']]
        x['movieId'], x.index = x.index, x['movieId'].values
        ratings['movieId'] = ratings['movieId'].map(x.to_dict()['movieId'])
        self.links['movieId'] = self.links['movieId'].map(x.to_dict()['movieId'])
        self.tags['movieId'] = self.tags['movieId'].map(x.to_dict()['movieId'])

        # Словарь тегов для каждого фильма
        self.movie_to_tags = self._preprocess_tags(max_tags)

        # Разделение на обучающую и тестовую выборки
        train_data = ratings.sample(frac=0.8, random_state=seed)
        test_data = ratings.drop(train_data.index)
        self.ratings = train_data if train else test_data

        # Создаем словарь для imdbId из title.ratings
        self.imdb_ratings = self._create_imdb_ratings_dict()

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        sample = self.ratings.iloc[idx]

        # Теги
        tags = self.movie_to_tags.get(sample['movieId'], [0] * 5)  # Паддинг нулями

        # Дополнительная информация по imdbId
        imdb_info = self.get_imdb_info(sample['movieId'])

        return {
            "user": torch.LongTensor([sample['userId']]),
            "movie": torch.LongTensor([sample['movieId']]),
            "rating": torch.FloatTensor([sample['rating']]),
            "tags": torch.LongTensor(tags),  # Теги как индексы
            "imdb_info": torch.FloatTensor(imdb_info)  # Дополнительные данные по imdbId
        }

    def _preprocess_tags(self, max_tags):
        """Преобразование тегов в индексы."""
        unique_tags = list(set(self.tags['tag'].values))
        tag_to_idx = {tag: idx + 1 for idx, tag in enumerate(unique_tags)}  # 0 для паддинга

        # Формируем словарь {movieId: [tag_idx, ...]}
        movie_to_tags = defaultdict(list)
        for _, row in self.tags.iterrows():
            movie_to_tags[row['movieId']].append(tag_to_idx[row['tag']])

        # Ограничиваем количество тегов
        for movie_id, tag_list in movie_to_tags.items():
            movie_to_tags[movie_id] = tag_list[:max_tags] + [0] * (max_tags - len(tag_list))

        return movie_to_tags

    def _create_imdb_ratings_dict(self):
        """Создаем словарь {tconst: [averageRating, num_ratings]} из файла title.ratings."""
        # Переименовываем столбец для удобства работы с ним
        self.title_ratings['tconst'] = self.title_ratings['tconst'].apply(lambda x: 'tt' + str(x)[2:])

        imdb_ratings = self.title_ratings[['tconst', 'averageRating', 'numVotes']]
        imdb_ratings_dict = imdb_ratings.set_index('tconst').to_dict('index')

        return imdb_ratings_dict

    def get_imdb_info(self, movie_id):
        """
        Получает информацию о фильме через tconst (например, рейтинг IMDB) из загруженного файла title.ratings.
        """
        # Извлекаем imdbId (tconst) из links.csv, добавляем 'tt', если его нет
        imdb_id = self.links.loc[self.links['movieId'] == movie_id, 'imdbId'].values
        if imdb_id:
            imdb_id = str(imdb_id[0])  # Форматируем как строку для обращения в словарь
            if not imdb_id.startswith('tt'):
                imdb_id = 'tt' + imdb_id[2:]  # Добавляем префикс 'tt', если его нет

            # Извлекаем данные из словаря
            imdb_data = self.imdb_ratings.get(imdb_id, {'averageRating': 0.0, 'numVotes': 0})
            imdb_rating = imdb_data['averageRating']
            num_votes = imdb_data['numVotes']
            
            # Возвращаем нужную информацию (можно добавить больше данных, если необходимо)
            return [imdb_rating, num_votes]
        else:
            return [0.0, 0]  # Если imdbId нет

    def add_new_user(self, seed=1, num_movies=20):
        """Добавление нового пользователя с его оценками."""
        new_user_id = self.ratings['userId'].max() + 1

        np.random.seed(seed)
        sampled_movie_ids = self.ratings['movieId'].unique()[:20]
        sampled_ratings = np.round(np.random.uniform(1.0, 5.0, num_movies) * 2) / 2

        new_user_ratings = pd.DataFrame({
            'userId': new_user_id,
            'movieId': sampled_movie_ids,
            'rating': sampled_ratings,
            'timestamp': int(pd.Timestamp.now().timestamp())
        })

        self.ratings = pd.concat([self.ratings, new_user_ratings], ignore_index=False)
        return new_user_id


In [96]:
from collections import defaultdict

batch_size = 200

sataset_source = r'.\ml-latest-small'

movielens_train = MovielensDataset(sataset_source, train=True)
movielens_test  = MovielensDataset(sataset_source, train=False)

train_loader = DataLoader(movielens_train, batch_size, True)
test_loader = DataLoader(movielens_test, batch_size, True)

In [97]:
device = torch.device('cpu' if torch.cuda.is_available() else 'cpu')
adeep_mf_model = newDeepFM().to(device)
adeep_mf_loss = nn.MSELoss()
adeep_mf_optimizer = torch.optim.Adam(adeep_mf_model.parameters(), lr=1e-3, weight_decay=1e-5)
ascheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(adeep_mf_optimizer, mode='min', factor=0.5, patience=2)

train(adeep_mf_model, train_loader, adeep_mf_loss, adeep_mf_optimizer, ascheduler, 15)

== Epoch 1 ==
loss: 2.538809  [  200/80669]
loss: 1.085818  [20200/80669]
loss: 1.169340  [40200/80669]
loss: 1.155993  [60200/80669]
loss: 0.999981  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 1.028674 

== Epoch 2 ==
loss: 0.972647  [  200/80669]
loss: 1.158235  [20200/80669]
loss: 1.071461  [40200/80669]
loss: 1.122660  [60200/80669]
loss: 0.990921  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.973830 

== Epoch 3 ==
loss: 0.967589  [  200/80669]
loss: 1.062046  [20200/80669]
loss: 1.154024  [40200/80669]
loss: 0.796481  [60200/80669]
loss: 1.141914  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.940766 

== Epoch 4 ==
loss: 1.101986  [  200/80669]
loss: 1.093272  [20200/80669]
loss: 0.905529  [40200/80669]
loss: 0.887550  [60200/80669]
loss: 0.984636  [80200/80669]
Test Error: 
 Accuracy: 0.0%, Avg loss: 0.920440 

== Epoch 5 ==
loss: 0.851863  [  200/80669]
loss: 0.912577  [20200/80669]
loss: 0.865105  [40200/80669]
loss: 0.971502  [60200/80669]
loss: 

In [117]:
import torch
import pandas as pd
import numpy as np

def get_movie_recommendations(model, user_id, dataset, n_recommendations=10, device='cuda'):
    """
    Получить рекомендации для конкретного пользователя.

    :param model: Обученная модель
    :param user_id: ID пользователя
    :param dataset: Экземпляр датасета (например, MovielensDataset)
    :param n_recommendations: Количество рекомендаций
    :param device: Устройство ('cuda' или 'cpu')
    :return: Список фильмов с рекомендованными рейтингами
    """
    model.eval()  # Переводим модель в режим оценки
    with torch.no_grad():
        # Получаем список всех фильмов
        all_movie_ids = dataset.movies['movieId'].values
        
        # Проверяем, какие фильмы пользователь уже оценил
        rated_movies = dataset.ratings[dataset.ratings['userId'] == user_id]['movieId'].values
        
        # Фильтруем фильмы, которые пользователь еще не оценил
        unseen_movies = [movie_id for movie_id in all_movie_ids if movie_id not in rated_movies]
        
        # Преобразуем индексы фильмов в индексы для эмбеддингов (если необходимо)
        movie_to_idx = {movie_id: idx for idx, movie_id in enumerate(all_movie_ids)}
        unseen_movie_indices = [movie_to_idx[movie_id] for movie_id in unseen_movies]

        # Получаем эмбеддинги для пользователя
        user_tensor = torch.LongTensor([user_id] * len(unseen_movie_indices)).to(device)
        movie_tensor = torch.LongTensor(unseen_movie_indices).to(device)
        
        # Получаем жанры фильмов (через one-hot encoding)
        genres = dataset.movies.loc[dataset.movies['movieId'].isin(unseen_movies), 'genres'].fillna('')
        all_genres = set(" ".join(genres).split())  # Получаем все уникальные жанры

        # Создаем one-hot encoding для жанров
        genres_tensor = torch.zeros(len(unseen_movies), len(all_genres))  # One-hot encoding для жанров
        for idx, genre_str in enumerate(genres):
            genre_list = genre_str.split('|')
            for genre in genre_list:
                if genre in all_genres:  # Проверяем, что жанр есть в списке всех жанров
                    genre_idx = list(all_genres).index(genre)
                    genres_tensor[idx, genre_idx] = 1
        genres_tensor = genres_tensor.to(device)

        # Получаем теги (предположим, что они строковые)
        tags_df = dataset.tags[dataset.tags['userId'] == user_id]
        tags = tags_df[tags_df['movieId'].isin(unseen_movies)]['tag'].fillna('')

        # Преобразуем теги в числовые представления (например, через one-hot encoding)
        tag_to_idx = {tag: idx for idx, tag in enumerate(set(" ".join(tags).split()))}
        tags_tensor = torch.LongTensor([torch.tensor([tag_to_idx.get(tag, -1) for tag in movie_tags.split()]) 
                                       for movie_tags in tags]).to(device)

        # Строим батч для вычисления предсказаний
        batch = {
            'user': user_tensor,
            'movie': movie_tensor,
            'genres': genres_tensor,
            'tags': tags_tensor
        }
        
        # Прогоняем через модель для получения рейтингов
        predictions = model(batch).cpu().numpy()
        
        # Выбираем топ-N фильмов по предсказанным рейтингам
        top_n_indices = np.argsort(predictions.flatten())[-n_recommendations:][::-1]
        
        # Создаем массив идентификаторов фильмов на основе полученных индексов
        top_n_movie_ids = [unseen_movies[i] for i in top_n_indices]
        top_n_ratings = predictions.flatten()[top_n_indices]

        # Выводим рекомендации с названиями фильмов и предсказанными рейтингами
        recommended_movies = []
        for movie_id, rating in zip(top_n_movie_ids, top_n_ratings):
            movie_title = dataset.movies.loc[dataset.movies['movieId'] == movie_id, 'title']
            if movie_title.empty:
                recommended_movies.append(f"Movie ID: {movie_id} не найден в списке фильмов.")
            else:
                movie_title = movie_title.values[0]
                recommended_movies.append(f"Movie ID: {movie_id}, Title: {movie_title}, Predicted Rating: {rating:.2f}")
        
        return recommended_movies

# Пример использования:
user_id = 611  # ID пользователя, для которого генерируем рекомендации
n_recommendations = 10  # Количество рекомендаций

# Получаем рекомендации
recommended_movies = get_movie_recommendations(deep_mf_model, user_id, movielens_train, n_recommendations, device='cpu')

# Выводим рекомендации
print(f"Топ-{n_recommendations} рекомендаций для пользователя {user_id}:")
for movie in recommended_movies:
    print(movie)


Топ-10 рекомендаций для пользователя 611:
Movie ID: 912, Title: Casablanca (1942), Predicted Rating: 4.51
Movie ID: 177593, Title: Three Billboards Outside Ebbing, Missouri (2017), Predicted Rating: 4.51
Movie ID: 858, Title: Godfather, The (1972), Predicted Rating: 4.49
Movie ID: 2324, Title: Life Is Beautiful (La Vita è bella) (1997), Predicted Rating: 4.47
Movie ID: 318, Title: Shawshank Redemption, The (1994), Predicted Rating: 4.47
Movie ID: 1221, Title: Godfather: Part II, The (1974), Predicted Rating: 4.46
Movie ID: 296, Title: Pulp Fiction (1994), Predicted Rating: 4.45
Movie ID: 4973, Title: Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001), Predicted Rating: 4.45
Movie ID: 44195, Title: Thank You for Smoking (2006), Predicted Rating: 4.45
Movie ID: 1245, Title: Miller's Crossing (1990), Predicted Rating: 4.44
